# Load and parse local dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
prompt_style = """下面是一个法律咨询问题，请提供一个回复来解决咨询问题，不需要提供思考过程。
### 指令：
你是一个法律咨询专家，请回答以下问题，不需要提供思考过程。

### 问题：
{}

### 回复:
{}"""

train_prompt_style = """下面是一个法律咨询问题，请提供一个回复来解决咨询问题，不需要提供思考过程。
### 指令：
你是一个法律咨询专家，请回答以下问题，不需要提供思考过程。

### 问题：
{}

### 回复:
{}"""

local_sft_data_path = ''


In [ ]:

def formatting_prompts_func(examples, eos_token):
    inputs = examples["question"]
    outputs = examples["answer"]
    texts = []
    for inputs, outputs in zip(inputs, outputs):
        text = train_prompt_style.format(inputs, outputs) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_dir)

model = AutoModelForCausalLM.from_pretrained(model_dir).to('cuda')
  

In [ ]:
%%time

dataset = load_dataset("json", data_files=local_sft_data_path, split="train")

processed_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    fn_kwargs={'eos_token': tokenizer.eos_token},
)

print('dataset example')
print(processed_dataset['text'][0])
